### Persistence -
    Persistence in LangGraph refers to the ability to save and restore the state of a workflow over time.

What is the importance of this ->

    1. Fault Tolerance  (Most IMP)
        As we save the memory of the state ... even if at some nodes the workflow fails -> we can restart from that very point as we know where we were at the last point
    2. Chatbot features 
        In chatbot an important features of it is to keep memory of the conversation.


### Checkpointers - 
Persistence in Langgraph is done using the concept of checkpointers 
    What checkpointers does is divide each of the graph flow in  form of checkpointes
    Every superstep becomes a checkpointers

![](img/checkpointers.png)
Here you can see 3 such superstep ... i.e 3 checkpointers to save the state in the memory

![](img/chekcpointer1.png)

### Threads in Persistence

    - Threads are used to store the session of the chat
    - Like you see in the chatgpt you have your chats stored in different sessions.
    - And when you open a certain session all the checkpointers fetched back as memory.
    - Now these are done by keeping a track with the thread_id created for each session.

### Make a joke exp agent

![](img/joke_exp.png)

In [3]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [12]:
load_dotenv()

model = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash"
)

In [13]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [14]:
def generate_joke(state: JokeState):
    prompt = f'Generate a joke on the topic - {state['topic']}'

    response = model.invoke(prompt).content

    return {
        'joke': response
    }

def generate_explanation(state: JokeState):
    prompt = f'Write an explanation for the joke - {state['joke']}'

    response = model.invoke(prompt).content
    
    return {
        'explanation': response
    }

In [15]:
# define the state
graph = StateGraph(JokeState)

# add nodes
graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

# add edges
graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

# compile the graph
workflow = graph.compile(checkpointer=checkpointer)

In [16]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic': 'pizza'},config=config1)

{'topic': 'pizza',
 'joke': "Why did the pizza maker go out of business?\n\nBecause he couldn't make enough **dough**!",
 'explanation': 'The humor in this joke comes from the **pun** on the word "dough."\n\nHere\'s why it\'s funny:\n\n1.  **Literal Meaning (in context of pizza):** When you think of a pizza maker, "dough" immediately refers to the mixture of flour, water, yeast, etc., that they use to make the pizza crust. If they couldn\'t make enough of *this* dough, they couldn\'t make enough pizzas.\n\n2.  **Slang Meaning (the punchline):** "Dough" is also a very common, informal slang term for **money**.\n\nThe joke plays on these two meanings. You expect the answer to be about the actual pizza dough, but the punchline cleverly switches to the financial meaning of "dough" as money. A business goes out of business because it can\'t make enough *money* (dough) to survive, not necessarily because it literally can\'t produce enough pizza dough.\n\nIt\'s a classic example of a **pun** 

In [17]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'tsunami'}, config=config2)

{'topic': 'tsunami',
 'joke': 'Tsunamis are serious natural disasters, and it\'s important to remember the devastating impact they have. Jokes about them can be sensitive.\n\nHowever, if we\'re going for a lighthearted, harmless play on words:\n\n> What do you call a very polite tsunami?\n> ... A "tsunami, if you please!"',
 'explanation': 'You\'re absolutely right to highlight the sensitivity around tsunamis and their devastating impact. It\'s crucial to remember that this joke is a very specific, lighthearted play on words that *deliberately avoids* any reference to the actual disaster, focusing purely on language.\n\nHere\'s the explanation for that particular wordplay joke:\n\n1.  **The Setup:** "What do you call a very polite tsunami?"\n    *   The key here is the word "polite," which immediately signals that the answer will involve a polite phrase.\n\n2.  **The Punchline:** "... A \'tsunami, if you please!\'"\n\n3.  **The Play on Words:**\n    *   The humor comes from combining t

In [11]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza get a job?\n\nBecause it wanted to make some *dough*!', 'explanation': 'This is a classic pun! Here\'s the explanation:\n\nThe joke relies on a **double meaning** of the word "dough."\n\n*   **Meaning 1 (Literal):** "Dough" is the primary ingredient in pizza. It\'s the flour, water, and yeast mixture that forms the base of the pizza.\n*   **Meaning 2 (Slang):** "Dough" is also a slang term for **money**.\n\nSo, the joke plays on the idea that the pizza, being made of dough, would want to get a job to earn *more dough* (money). It\'s a humorous and unexpected connection between what a pizza is made of and what people work for.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f104571-e48a-6f96-8002-dadb4584361f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-02-07T18:59:24.194164+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint

### Benefits of Persistence

![](img/benefit_of_persistence.png)

HOW THIS HAPPENS JUST WATCH THE LEC AND ADD THE NOTES LATER 